# Pipeline to Detect Lanes
---

## Calibrate Camera

This step needs to only be done once.

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import pickle
from lane_functions import chessboard_corners, find_reclanes, draw_box, window_mask, find_sliding_poly, draw_lane

from IPython.display import Image, HTML, display
from glob import glob
%matplotlib inline

## Parameters and Cal Data

These include:
- Checkerboard dimensions
  + This took me an especially long time to debug. It turns out, I had
    miscounted the number of corners for each of the checkerboards.
    The count in the original lab example had (8,5). In contrast, the
    checkerboards here have (9,6) corners.
- Input file directory with images
  + This is found in `camera_cal/`.

In [ ]:
# Parameters to find the chessboard corners. We have 9 columns and 6 rows, stored in "camera_cal"
checkcols = 9
checkrows = 6
imdir = 'camera_cal/'

# Find the object points and correspond with image points
objpoints, imgpoints = chessboard_corners( imdir, checkcols, checkrows )

# Visualize the images
imagesList=''.join( ["<img style='width: 150px; margin: 0px; float: left; border: 1px solid black;' src='%s' />" % str(s) 
                 for s in sorted(glob('examples/calibration_output/*.jpg')) ])
display(HTML(imagesList))

## Find Parameters to Undistort

Make calls to calibrate the camera. The calibration and distortion parameters are stored in `mtx` and `dist`.

In [ ]:
# Test undistortion on an image
img = cv2.imread('camera_cal/calibration1.jpg')
img_size = (img.shape[1], img.shape[0])

# Do camera calibration given object points and image points
print('Length Objects: {}, Image Points: {}'.format(len(objpoints), len(imgpoints)))
print('Image Size: {}'.format(img_size))
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)
dst = cv2.undistort(img, mtx, dist, None, mtx)

# Visualize undistortion
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10)); 
ax1.axis('off'); ax2.axis('off')
ax1.imshow(img); plt.imsave('examples/distorted.jpg',img); 
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(dst); plt.imsave('examples/undistorted.jpg', dst); 
ax2.set_title('Undistorted Image', fontsize=30)

## Reprojection Parameters (Perspective)

Warp the image to an overhead perspective

In [ ]:
# Format is:
#    [ UpperLeft, LowerLeft, LowerRight, UpperRight ]
#    Assuming some symmetry, I added equal numbers left and right
src = np.float32(
     [[img_size[0] / 2 - 64, img_size[1] / 2 + 100],
      [img_size[0] / 6 - 10, img_size[1]],
      [img_size[0] * 5 / 6 + 10, img_size[1]],
      [img_size[0] / 2 + 64, img_size[1] / 2 + 100]])
dst = np.float32(
    [[(img_size[0] / 4) + 50, 0],
     [(img_size[0] / 4) + 50, img_size[1]],
     [(img_size[0] * 3 / 4) - 50, img_size[1]],
     [(img_size[0] * 3 / 4) - 50, 0]])
M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst,src)

In [ ]:
img = cv2.imread('test_images/test2.jpg')
# img = cv2.imread('test_images/straight_lines2.jpg')
calimg = cv2.undistort(img, mtx, dist, None, mtx)
warped = cv2.warpPerspective(calimg, M, img_size, flags=cv2.INTER_LINEAR)

f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20,10))
ax1.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
ax1.set_title('Original Image', fontsize=30); ax1.axis('off')
ax2.imshow(cv2.cvtColor(calimg, cv2.COLOR_BGR2RGB))
ax2.set_title('Undistorted Image', fontsize=30); ax2.axis('off')
ax3.imshow(cv2.cvtColor(warped, cv2.COLOR_BGR2RGB))
ax3.set_title('Warped Image', fontsize=30); ax3.axis('off')

## Draw lines on the image

In [ ]:
srcpairs = []; dstpairs = []
for pair in src:
    srcpairs += [tuple(pair.astype(int))]
for pair in dst:
    dstpairs += [tuple(pair.astype(int))]
    
fig = plt.figure(figsize=(40,40)); plt.axis('on'); fig.subplots(1,3)
ax1 = plt.subplot(1,3,1); ax1.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
ax1.set_title('Original Image', fontsize=30); 
ax2 = plt.subplot(1,3,2); draw_box(calimg, srcpairs)
ax2.set_title('Undistorted Image', fontsize=30); 
ax3 = plt.subplot(1,3,3); draw_box(warped, dstpairs)
ax3.set_title('Warped Image', fontsize=30); 

## Do some line detection from the overhead imagery

In [ ]:
color_binary, combined_binary = find_reclanes(warped)

# Plotting thresholded images
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.set_title('Stacked thresholds')
ax1.imshow(color_binary)

ax2.set_title('Combined S channel and gradient thresholds')
ax2.imshow(combined_binary, cmap='gray')

In [ ]:
left_fit, right_fit = find_sliding_poly(combined_binary)

In [ ]:
binary_warped = combined_binary

# Generate x and y values for plotting
ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

# out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
# out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
# plt.imshow(out_img)
plt.imshow(binary_warped)
plt.plot(left_fitx, ploty, color='red')
plt.plot(right_fitx, ploty, color='red')
plt.xlim(0, 1280)
plt.ylim(720, 0)

In [ ]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None

In [ ]:
# Create an image to draw the lines on
warp_zero = np.zeros_like(warped).astype(np.uint8)

# color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
color_warp = np.copy(warp_zero)

# # Recast the x and y points into usable format for cv2.fillPoly()
pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
pts = np.hstack((pts_left, pts_right))

# Draw the lane onto the warped blank image
cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

# Warp the blank back to original image space using inverse perspective matrix (Minv)
newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
# Combine the result with the original image
result = cv2.addWeighted(calimg, 1, newwarp, 0.3, 0)
plt.imshow(cv2.cvtColor(result,cv2.COLOR_BGR2RGB))

In [ ]:
def detect_lanes(imname, mtx, dist, M, Minv):
    '''
    Draw lanes on top of image given distortion and image warping matrix
    Arguments:
      imname - image name (text)
      mtx - camera calibration coefficient
      dist - distortion coefficient
      M - warping matrix
      Minv = inverse of warping matrix
      
    Returns:
      image with lanes overlayed on top
    '''
    if type(imname) == str:
        img = cv2.imread(imname)
    else:
        img = imname
    calimg = cv2.undistort(img, mtx, dist, None, mtx)
    warped = cv2.warpPerspective(calimg, M, img_size, flags=cv2.INTER_LINEAR)
    color_binary, combined_binary = find_reclanes(warped)
    left_fit, right_fit = find_sliding_poly(combined_binary)
    lanedrawn = draw_lane(left_fit, right_fit, img_size=img_size)
    newwarp = cv2.warpPerspective(lanedrawn, Minv, (img.shape[1], img.shape[0]))     
    return cv2.addWeighted(calimg, 1, newwarp, 0.3, 0)

laneimage = detect_lanes('test_images/test1.jpg', mtx, dist, M, Minv)

plt.imshow(cv2.cvtColor(laneimage,cv2.COLOR_BGR2RGB))

In [ ]:
%load_ext autoreload
%autoreload 2

# Create a decorator function that will process a single frame with known calibration and warp parameters
def detect_lanes_video(img):
    return detect_lanes(img, mtx, dist, M, Minv)

In [ ]:
from moviepy.editor import VideoFileClip

video_output1 = 'output_videos/challenge_video_output.mp4'
video_input1 = VideoFileClip('input_videos/challenge_video.mp4')#.subclip(22,26)
processed_video = video_input1.fl_image(detect_lanes_video)
%time processed_video.write_videofile(video_output1, audio=False)